<a href="https://colab.research.google.com/github/neil51/SO/blob/main/exercise_01_tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Tensors

In chapter *1 - Introduction* of the lecture, we recapitulated tensor notation and tensor analysis. In this exercise, we will gain some more confidence in working with tensors on paper, but also in code. 

To solve the tasks with code, we will use PyTorch, a powerful Python package to operate on tensors. In comparison to NumPy, it stores gradients together with tensors and thus allows automatic differentiation. The package is used widely for machine learning and optimization. For installation it is best to create a new conda environement via
```
    conda create -n pytorch python=3.10
```
we can than activate that environment
```
    conda activate pytorch
``` 
and then use 
```
    conda install matplotlib pytorch torchvision jupyter jupyter-lab
```
to install the required packages. After that, you should be able to import the torch package in a Jupyter Notebook:

In [3]:
import torch
torch.set_default_dtype(torch.double)

## Task 1 - Vector products

Given two vectors $\mathbf{a}, \mathbf{b} \in \mathcal{R}^3$ with
$$
\mathbf{a} = \begin{pmatrix}2\\1\\3\end{pmatrix} \quad \mathbf{b} = \begin{pmatrix}5\\0\\1\end{pmatrix}
$$
define the vectors in torch and compute the scalar product, cross product and outer product. 

In [5]:
# --> define a 
a = torch.Tensor([2.0, 1.0, 3.0])

# --> define b
b = torch.Tensor([5.0, 0.0, 1.0])

In [ ]:
# print a
print(a)

tensor([2., 1., 3.])


In [ ]:
# print b
print(b)

tensor([5., 0., 1.])


In [ ]:
print("Inner product:")
# --> compute inner product
torch.inner(a, b)

Inner product:


tensor(13.)

In [ ]:
print("Cross product:")
# --> compute cross product
torch.cross(a, b)

Cross product:


tensor([ 1., 13., -5.])

In [ ]:
print("Outer product:")
# --> compute outer product
torch.outer(a, b)

Outer product:


tensor([[10.,  0.,  2.],
        [ 5.,  0.,  1.],
        [15.,  0.,  3.]])

## Task 2 - Tensor products
Given the tensors $\mathbf{A}, \mathbf{B} \in \mathcal{R}^{3 \times 3}$ and $\mathbb{C} \in \mathcal{R}^{3 \times 3 \times 3 \times 3}$ convert the following expressions to sums of components and determine the dimensions of the resulting tensor. 

Example: 

$$\mathbf{A} \cdot \mathbf{b} \rightarrow \sum_{i,j} A_{ij}b_j \mathbf{e}_j$$

a)  $$\mathbf{a} \cdot \mathbf{A} \cdot \mathbf{b}$$
b) $$\mathbf{b} \cdot \mathbf{A} \cdot \mathbf{a}$$
c) $$\mathbf{A} \cdot \mathbf{B} \cdot \mathbf{b}$$
d) $$(\mathbf{A} : \mathbf{B}) \mathbf{b}$$
e) $$(\mathbf{a} \otimes \mathbf{b}) : \mathbf{B}$$
f) $$\mathbf{A} \otimes \mathbb{C} : \mathbf{B}$$


> Calculate results by hand

Convert the following expressions to symbolic notation and determine the dimensions of the resulting tensor: 

g) $$\sum_{z,j} A_{zj}b_z \mathbf{e}_j$$
h) $$\sum_{i,j,k} A_{ij}B_{jk}a_k \mathbf{e}_i$$
i) $$\sum_{m,n,o,p,i} C_{mnop}A_{po}\delta_{ni}a_{i} \mathbf{e}_m$$

> Calculate results by hand

Given the values 
$$
\mathbf{A} = 
\begin{pmatrix}
    6 & 2 & 1\\
    4 & 7 & 6\\
    0 & 2 & 9
\end{pmatrix} 
\quad 
\mathbf{B} = 
\begin{pmatrix}
    5 & 7 & 11\\
    0 & 4 & 3\\
    1 & 2 & 9
\end{pmatrix}
\quad 
C_{ijkl} = 1 \forall i,j,k,l
$$
define the tensors in torch and compute the expressions above. Reuse $\mathbf{a}$ and $\mathbf{b}$ from the first task.

*Tips:* 
- What we denote with $\cdot$ in the lecture, can be written with an `@` or `torch.tensordot(...,dim=1)`  in numpy and torch.
- What we denote with $:$ in the lhe lecture, can be written with `torch.tensordot` in numpy and torch.
- Multiplication between scalars is done simply by `*`.
- We can use `torch.einsum()` to define arbitrary expressions using Einstein's summation convention. Here, the function automatically sums over indices in an expression, e.g. `torch.einsum("ij,j->i",A,b)` computes $\sum_{ij} A_{ij}b_j \mathbf{e}_i$

In [13]:
# --> Define tensors A, B, C
A = torch.Tensor([[6., 2., 1.], [4., 7., 6.], [0., 2., 9.]])
B = torch.Tensor([[5., 7., 11.], [0., 4., 3.], [1., 2., 9.]])
C = torch.ones(3,3,3,3)
I = torch.eye(3)

# --> Compute products a) to f)
print(a @ A @ b)
print(torch.tensordot(A,B) * b)
print(torch.tensordot(torch.outer(a,b), B))
print(torch.einsum("ij,klmn,nm", A,C,B).shape)

# --> Compute products g) to i)
print(torch.einsum("zj,z", A, b))
print(torch.einsum("ij, jk, k", A, B, a))
print(torch.einsum("mnop, po, ni, i", C, A, I, a))

tensor(115.)
tensor([930.,   0., 186.])
tensor(117.)
torch.Size([3, 3, 3, 3])
tensor([30., 12., 14.])
tensor([357., 477., 305.])
tensor([222., 222., 222.])


## Task 3 - Gradients
Given the vectorfield $f: \mathcal{R}^2 \rightarrow \mathcal{R}$ defined as 

$$
f(\mathbf{x}) = (\mathbf{x} - \tilde{\mathbf{x}}) \cdot \mathbf{Q} \cdot (\mathbf{x} - \tilde{\mathbf{x}})
$$
with 
$$
\mathbf{Q} = 
\begin{pmatrix}
    2 & 1 \\
    1 & 1 
\end{pmatrix} 
\quad 
\text{and}
\quad
\tilde{\mathbf{x}} = 
\begin{pmatrix}
    -1\\
    1 
\end{pmatrix}
$$
compute the gradient analytically.

> Compute the gradient by hand

Doing these computations by hand takes a while. Therefore we take a look at how to compute gradients using PyTorch. To do so, we start by defining $\mathbf{Q}$, $\tilde{\mathbf{x}}$ and the function $f(\mathbf{x})$. The function $f(\mathbf{x})$ can be implemented in a straight forward way and you should try a straight forward implementation first. 

However, we would like to be able to evaluate the function for many values of $\mathbf{x}$ at the same time. This is equivalent to passing a tensor of the shape $\mathcal{R}^{... \times 2}$ with arbitray dimensions except the last axis. This can be implemented using an ellipsis `...` in `torch.einsum()`.

In [ ]:
# --> Define Q and x_tilde 

# --> Define the fucntion f(x)


If your function is defined correctly, the following cell should plot the function values as a contour plot. It produces an error now, because the function f(x) has not been defined yet.

In [ ]:
from utils import plot_contours

# Define x
x0 = torch.linspace(-3, 3, steps=100, requires_grad=True)
x1 = torch.linspace(-3, 3, steps=100, requires_grad=True)
x = torch.stack(torch.meshgrid(x0, x1, indexing="xy"), dim=2)

plot_contours(x[...,0], x[...,1], f(x), title="f(x)")


NameError: name 'f' is not defined

Note that the `requires_grad=True` argument defines that these specific tensors will be used in gradient computations. They reserve storage for the tensor data as well as the gradients. Now, lets compute the actual gradients with automatic differentiation:

In [ ]:
dfdx = torch.autograd.grad(f(x).sum(), x)[0]

plot_contours(x[..., 0], x[..., 1], dfdx[..., 0], title="Gradient in x_0")
plot_contours(x[..., 0], x[..., 1], dfdx[..., 1], title="Gradient in x_1")


NameError: name 'f' is not defined